In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from keras import regularizers
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import pickle
from sklearn.metrics import confusion_matrix
import GetPickles
import var
import Viz
from keras.applications.vgg16 import VGG16, preprocess_input

Using plaidml.keras.backend backend.


In [2]:
def get_vgg16(dim):
    model = Sequential()
    baseModel = VGG16(weights="imagenet", include_top=False,
        input_tensor=Input(shape=dim))
    
    model.add(baseModel)
    headModel = model.add(AveragePooling2D(pool_size=(7, 7)))
    headModel = model.add(Flatten(name="flatten"))
    headModel = model.add(Dense(128, activation="relu"))
    headModel = model.add(Dropout(0.3))
    headModel = model.add(Dense(3, activation="softmax", name = 'Output'))

    # place the head FC model on top of the base model (this will become
    # the actual model we will train)


    # loop over all layers in the base model and freeze them so they will
    # *not* be updated during the first training process
    for layer in baseModel.layers:
        layer.trainable = False
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model


In [3]:
x_train, x_test, y_train, y_test = GetPickles.get_samples('vgg16')
print(x_train.shape, x_test.shape) 
print(y_train.shape, y_test.shape)

(6512, 224, 224, 3) (724, 224, 224, 3)
(6512, 3) (724, 3)


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator


augment = False 

early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=18)
model_checkpoint = ModelCheckpoint('../models/VGG16-ModelCheckpointWeights.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 350
batch_size = 32
if var.img_type == 'grey': 
    dim = (var.vgg_dimension[0], var.vgg_dimension[1], 1)
else: 
    dim = (var.vgg_dimension[0], var.vgg_dimension[1], 3)
    
vgg_16 = get_vgg16(dim =dim)

if augment: 
    augmentation =ImageDataGenerator(rotation_range = 15, width_shift_range = .1, height_shift_range = .1, 
                                                           horizontal_flip = True, fill_mode = 'nearest')
    augmentation.fit(x_train)
    vgg_16_history = vgg_16.fit_generator(augmentation.flow(x_train, y_train, batch_size = batch_size),
                epochs = epochs, steps_per_epoch = 8,
         callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)
else: 
    
    vgg16_history = vgg_16.fit(x_train, y_train, batch_size = batch_size,
                epochs = epochs, 
         callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)


INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


Train on 6512 samples, validate on 724 samples
Epoch 1/350
6512/6512 [==============================] - 188s 29ms/step - loss: 0.6359 - acc: 0.7320 - val_loss: 0.5378 - val_acc: 0.7970

Epoch 00001: val_loss improved from inf to 0.53780, saving model to ../models/VGG16-ModelCheckpointWeights.h5
Epoch 2/350
6512/6512 [==============================] - 168s 26ms/step - loss: 0.4976 - acc: 0.8028 - val_loss: 0.4858 - val_acc: 0.8080

Epoch 00002: val_loss improved from 0.53780 to 0.48584, saving model to ../models/VGG16-ModelCheckpointWeights.h5
Epoch 3/350
6512/6512 [==============================] - 168s 26ms/step - loss: 0.4540 - acc: 0.8182 - val_loss: 0.4514 - val_acc: 0.8218

Epoch 00003: val_loss improved from 0.48584 to 0.45136, saving model to ../models/VGG16-ModelCheckpointWeights.h5
Epoch 4/350
6512/6512 [==============================] - 168s 26ms/step - loss: 0.4297 - acc: 0.8254 - val_loss: 0.4360 - val_acc: 0.8204

Epoch 00004: val_loss improved from 0.45136 to 0.43597, sav

6512/6512 [==============================] - 168s 26ms/step - loss: 0.2420 - acc: 0.9085 - val_loss: 0.3296 - val_acc: 0.8826

Epoch 00038: val_loss did not improve from 0.32826
Epoch 39/350
6512/6512 [==============================] - 168s 26ms/step - loss: 0.2424 - acc: 0.9062 - val_loss: 0.3287 - val_acc: 0.8826

Epoch 00039: val_loss did not improve from 0.32826
Epoch 40/350
6512/6512 [==============================] - 168s 26ms/step - loss: 0.2420 - acc: 0.9083 - val_loss: 0.3257 - val_acc: 0.8798

Epoch 00040: val_loss improved from 0.32826 to 0.32568, saving model to ../models/VGG16-ModelCheckpointWeights.h5
Epoch 41/350
6512/6512 [==============================] - 168s 26ms/step - loss: 0.2416 - acc: 0.9082 - val_loss: 0.3237 - val_acc: 0.8743

Epoch 00041: val_loss improved from 0.32568 to 0.32369, saving model to ../models/VGG16-ModelCheckpointWeights.h5
Epoch 42/350
6512/6512 [==============================] - 168s 26ms/step - loss: 0.2381 - acc: 0.9083 - val_loss: 0.3250 - 

### Loss and Accuracy

In [ ]:
Viz.plot_loss_accuracy(vgg16_history)

### ROC & AUC

In [ ]:
if var.img_type == 'grey': 
    dim = (var.vgg_dimension[0], var.vgg_dimension[1], 1)
else: 
    dim = (var.vgg_dimension[0], var.vgg_dimension[1], 3)
    
vgg_16 = get_vgg16(dim =dim)
vgg_16.load_weights('../models/VGG16-ModelCheckpointWeights.h5')
Viz.plot_roc_auc(vgg_16, x_test, y_test)

### Confusion Matrix

In [ ]:
if var.img_type == 'grey': 
    dim = (var.vgg_dimension[0], var.vgg_dimension[1], 1)
else: 
    dim = (var.vgg_dimension[0], var.vgg_dimension[1], 3)
    
vgg_16 = get_vgg16(dim =dim)
vgg_16.load_weights('../models/VGG16-ModelCheckpointWeights.h5')





y_test_prob = [np.argmax(i) for i in vgg_16.predict(x_test)]
y_test_labels = [np.argmax(
    i) for i in y_test]
test_cnf = confusion_matrix(y_test_labels, y_test_prob)

y_train_prob = [np.argmax(i) for i in vgg_16.predict(x_train)]
y_train_labels = [np.argmax(i) for i in y_train]
train_cnf = confusion_matrix(y_train_labels, y_train_prob)

Viz.plot_model_cm(test_cnf, train_cnf, classes = ['No Weapon', 'Handgun', 'Rifle'])


### Test on Google

In [ ]:
import cv2
from lime import lime_image
from skimage.segmentation import mark_boundaries

In [ ]:
base_path = '../TestImages'
if var.img_type == 'grey': 
    dim = (var.vgg_dimension[0], var.vgg_dimension[1], 1)
else: 
    dim = (var.vgg_dimension[0], var.vgg_dimension[1], 3)
    
vgg_16 = get_vgg16(dim =dim)
vgg_16.load_weights('../models/VGG16-ModelCheckpointWeights.h5')


for file in os.listdir(base_path): 
    if file == '.ipynb_checkpoints':
        continue
    full_path = f'{base_path}/{file}'
    img = GetPickles.get_image_value(full_path, var.vgg_dimension, var.img_type)
    img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])
    pred = vgg_16.predict(img)[0]
    print(f'{file}\t\t{np.argmax(pred)}\t\t{pred.max()}\t\t{pred}')